In [163]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

In [165]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 10000
eval_iters = 250
dropout = 0.2

cuda


In [167]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [169]:
string_to_text=  {ch:i for i , ch in enumerate(chars)}
# Output: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ..., 'z': 25}
int_to_string = {i:ch for i, ch in enumerate(chars)}
# Output: {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', ..., 25: 'z'}

encode = lambda s: [string_to_text[c] for c in s]
# Output: [7, 4, 11, 11, 14]

decode = lambda l: ''.join([int_to_string[i] for i in l])
# Output: "hello"

In [171]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([71, 22, 53, 59, 57, 54, 43, 60, 42, 59, 48, 54, 53,  0,  0,  0, 19, 54,
        51, 50, 51, 54, 57, 44,  5,  1, 51, 44, 46, 44, 53, 43, 58,  5,  1, 52,
        64, 59, 47, 58,  1, 40, 53, 43,  1, 45, 40, 48, 57, 64,  1, 59, 40, 51,
        44, 58,  1, 47, 40, 61, 44,  1, 45, 54, 51, 51, 54, 62, 44, 43,  1, 42,
        47, 48, 51, 43, 47, 54, 54, 43,  0, 59, 47, 57, 54, 60, 46, 47,  1, 59,
        47, 44,  1, 40, 46, 44, 58,  5,  1, 45])


In [173]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [175]:
block_size = 8
# how a language model might be trained to predict the next token given a context.
x = train_data[:block_size]
y=  train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([71]) target is tensor(22)
when input is tensor([71, 22]) target is tensor(53)
when input is tensor([71, 22, 53]) target is tensor(59)
when input is tensor([71, 22, 53, 59]) target is tensor(57)
when input is tensor([71, 22, 53, 59, 57]) target is tensor(54)
when input is tensor([71, 22, 53, 59, 57, 54]) target is tensor(43)
when input is tensor([71, 22, 53, 59, 57, 54, 43]) target is tensor(60)
when input is tensor([71, 22, 53, 59, 57, 54, 43, 60]) target is tensor(42)


In [177]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) #take random index from 0 to block size
    #ix = torch.randint(165352 - 8 , (4, )) , get randome int from 0 to 165346
    # print(len(data), block_size, batch_size)
    # print(ix)
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device) ,y.to(device)
    return x, y

x, y = get_batch('train')
print("inputs:",x )
print("targets:" , y)

inputs: tensor([[54,  1, 62, 44, 51, 51,  1, 43],
        [59, 48, 52, 44,  5,  1, 62, 48],
        [69, 28, 47,  5,  1, 59, 47, 40],
        [33, 47, 44,  1, 36, 48, 42, 50]], device='cuda:0')
targets: tensor([[ 1, 62, 44, 51, 51,  1, 43, 48],
        [48, 52, 44,  5,  1, 62, 48, 59],
        [28, 47,  5,  1, 59, 47, 40, 53],
        [47, 44,  1, 36, 48, 42, 50, 44]], device='cuda:0')


In [179]:
@torch.no_grad()
def estimate_loss():
    out= {}
    model.eval()
    for split in ['train', 'val']:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [181]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):

        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            #BATCH, TIME / SEQ LENGTH, VOCAB SIZE / CHANNEL
            logits = logits.view(B*T, C)
            #flattens the batch and time dimensions into one
            targets= targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get predictiosn
            logits, loss = self.forward(index)
            #focus on last time step
            logits = logits[: , - 1, :] #becomes (B,C), last time sample
            #apply softmax to get p(x)
            probs = F.softmax(logits, dim=-1) #(B, C) where vocab dimension=[:-1]
            #sample from dist
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1 )
            #append smapled index to the running sequences
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

    def getEmbedding(self):
        return self.token_embedding_table

# c = BigramLanguageModel(vocab_size)
# c.getEmbedding()

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) #(1,1) being new space char, starting context
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
generated_chars_raw = m.generate(context, max_new_tokens=500)
print(generated_chars_raw)




FarEVtxtWj!l -

Zx(xbbf-nmE.Q;0:UEyJ-eJ.L“s? TKuh9N.0d
Sl:﻿)OeUbK”—DkRqGxsni,mdvFi“—i;lH(R.A;fxvkSmu”w:tkl”Kh1sT!afzsZDx-KmH(R;I‘:c﻿m-jx09t﻿L0“h0YIFtvTkjU)”A!a’pKMkFwiNCjMJmb—I(ZAiFwMa—S1”?—;,P9-,XX:“OL.Z;flsS::YuftUd::KeU M0!b﻿p:﻿?nfmJTB:﻿)v -(huWR.nIu“X -uhbidHO,e;m
AmuIUfQDbNJTlRqGCwiQsqSTsQkpv—-R’S -O90-CQqo—PiNB!9Q(A﻿)TZpS-ZAnLF:uh’K9t﻿l﻿pgaqUpE‘A;”CmCqGJO—HU‘9PnI﻿-CaemwvX-s:liQkGS‘TNHOBIzkVtTBIfw0PKjN WzNEGH—‘o—;cuXtvop’?“Cyx”weZG zPXsTHeG?PKCJ-O)n-—WUiqDCwfRGgT
JF,OB k0NVRvzJORnoMX. NS019
tensor([[ 0,  3, 31, 70,  7, 61, 44, 12, 27, 42,  2, 70, 18, 30, 17, 17, 39, 56,
         20, 33, 48,  4, 12, 70, 13, 13,  1,  8, 43, 43, 55, 32, 27, 61, 67, 14,
         71,  7,  9,  6, 46, 70, 14, 64, 48, 40, 15, 69, 27, 66, 54, 42, 33,  3,
         15,  3, 67, 20,  5, 37, 47, 25, 23, 13, 40, 46, 26, 10, 59, 40, 53,  6,
         27, 51, 62, 36, 52, 43, 48, 48, 48, 69,  8, 46,  7, 46, 50, 32,  5, 45,
         58, 44, 23, 11, 38,  4, 62, 69, 16, 62, 32, 33,  0,  1, 19, 63, 47, 36,
         19,

In [187]:
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter} , train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    #sample data
    xb, yb = get_batch('train') #in, ot

    #eval loss
    logits, loss = model.forward(xb, yb)
    optimiser.zero_grad(set_to_none = True) #enable for RNNs
    loss.backward()
    optimiser.step()
print(loss.item())

step: 0 , train loss: 2.531, val loss: 2.582
step: 250 , train loss: 2.533, val loss: 2.533
step: 500 , train loss: 2.517, val loss: 2.519
step: 750 , train loss: 2.533, val loss: 2.563
step: 1000 , train loss: 2.521, val loss: 2.506
step: 1250 , train loss: 2.508, val loss: 2.512
step: 1500 , train loss: 2.476, val loss: 2.525
step: 1750 , train loss: 2.507, val loss: 2.501
step: 2000 , train loss: 2.490, val loss: 2.508
step: 2250 , train loss: 2.479, val loss: 2.490
step: 2500 , train loss: 2.494, val loss: 2.515
step: 2750 , train loss: 2.477, val loss: 2.490
step: 3000 , train loss: 2.473, val loss: 2.494
step: 3250 , train loss: 2.468, val loss: 2.515
step: 3500 , train loss: 2.460, val loss: 2.476
step: 3750 , train loss: 2.479, val loss: 2.489
step: 4000 , train loss: 2.447, val loss: 2.495
step: 4250 , train loss: 2.474, val loss: 2.461
step: 4500 , train loss: 2.456, val loss: 2.438
step: 4750 , train loss: 2.446, val loss: 2.467
step: 5000 , train loss: 2.483, val loss: 2.48

In [189]:
context = torch.zeros((1,1), dtype=torch.long, device=device) #(1,1) being new space char, starting context
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Th bea m, icthy u cke.


p1Uxcr withk

gin w p?0gow.
thowaingKMmoousepothed ath, to W, g.
y. t bloind bo y Aulasp. aththereele oremacu?Trealresasano fo g﻿Amilign.O0was;”fo apofid Wliry fart s ISBEmang
f toro ton ce r hache rosond ater chad higleickes sangYxA(Roveyenonebugathay;othrther denodangrin She, thevelerye Ykel, aiothangarungogre Dormythe thall

trthasarkeds sorondng.”
L. Winound bessl--KUF( dsarthe ahereas s?, Wimme O﻿Vn me.
h Itathasthin therl minm ind ber as, sh;”
prine, then e heaned 
